In [1]:
import nltk
import torch
import numpy as np

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/nithinrao/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
from InferSent.models import InferSent 

In [5]:
# Load model
model_version = 2
MODEL_PATH = "encoder/infersent%s.pkl" % model_version
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': model_version}
model = InferSent(params_model)
model.load_state_dict(torch.load(MODEL_PATH))

# If infersent1 -> use GloVe embeddings. If infersent2 -> use InferSent embeddings.
W2V_PATH = 'GloVe/glove.840B.300d.txt' if model_version == 1 else 'fastText/crawl-300d-2M.vec'
model.set_w2v_path(W2V_PATH)

# Load embeddings of K most frequent words
model.build_vocab_k_words(K=100000)

Vocab size : 100000


In [6]:
print(model)

InferSent(
  (enc_lstm): LSTM(300, 2048, bidirectional=True)
)


In [7]:
# Load some sentences
sentences = []
with open('train/neutral.txt') as f:
    for line in f:
        sentences.append(line.strip())
print(len(sentences))

365528


In [8]:
embeddings = model.encode(sentences[:10], bsize=128, tokenize=True, verbose=True)
print('nb sentences encoded : {0}'.format(len(embeddings)))

Nb words kept : 155/155 (100.0%)
Speed : 14.3 sentences/s (cpu mode, bsize=128)
nb sentences encoded : 10


In [9]:
embeddings.shape

(10, 4096)

In [10]:
type(embeddings)

numpy.ndarray

In [ ]:
model.encode()